# **Delta tables**

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession, types

spark = SparkSession.builder.config("spark.driver.host", "localhost").getOrCreate()

#to build session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rdddf').master('local').getOrCreate()

### **Note - Run commands on databricks**

In [ ]:
# pip install delta-spark

In [ ]:
from delta.tables import *

### **creating datalake table**

In [ ]:
DeltaTable.create(spark)\
.tableName("delta_internal_demo")\
.addColumn("emp_id","INT")\
.addColumn("emp_name","STRING")\
.addColumn("makrs","INT")\
.property("description","table craeted for demo purpose")\
.location("dbfs:/FileStore/delta")\
.execute()

**On execution**
* In data - DAatbase Tables a table is created with name delta_internal_demo
* In DBFS - at dbfs:/FileStore/delta" fiel is created with name _delta_log
* If we insert some data to table than that data file will be also stored in same location.
* Every operation that we execute on the delta table will be recorded in the _delta_log folder.

In [ ]:
%fs
ls dbfs:/FileStore/delta/_delta_log

**This will list all the files present at the location mentioned.**
* within this there will be json file to store the logs
* crc file Cyclic redundant check
* some other files.

In [ ]:
%fs
head dbfs:/delta/_delta_log/00000000000000000000.json

To read the json log file.
* as we have craeted delta table so this file craete info about delta table creation operation.
* Miainly info about metadata of table created. column anme, type etc.
* who created, when created, about user info also.

In [ ]:
%sql
select * from delta_internal_demo;

**Insert data into table**

In [ ]:
%sql
insert into delta_internal_demo values(100,"ab",70)

on execution of command on delta table a new json log file is created in sequence.like 00000000000000000001.json means its 2nd log file created.

**note - data file will also created in delta folder. In the form of parquet file**

### **To read the content of the log file**

In [ ]:
%fs
head dbfs:/delta/_delta_log/00000000000000000001.json

This file is based on current insertion operation of data into delta table.
* {"add":{"path":"part-00000-4dc3c8a4-6a1a-46ee-97ba-c75f7c443ff4-c000.snappy.parquet"
  * add - it shows add operation is dome
  * path - data file name. The insertion of data created a data file.
  * contain ither info aslo like size, modification time, number of record affected, min value, max value of each column.

In [ ]:
%sql
insert into delta_internal_demo values(200,"philip",65);
insert into delta_internal_demo values(300,"Lara",68);


*  Each insertion operation is considered seprate operation and thus for each a new log file is created.
*  after 10 log file a parquet file is created which will contian info about the 10 saved log files.
   *  d00000000000000000010.checkpoint.parquet - this is file created after 10 log files.
*  For each insertion a datafile is created.
*  In log file of deletion operation
   *  Operation - remove
   *  path - data file name tht should not be considered during select query or for user output.When engine will read this file than it will get understand not to read these data files.
*  '    

**soft delete pattern**
* Which means even we are not intersted in particualr data file it will not be removed immidiately. there is deafault time period for this of 7 days after that it will be deleted. But we can configure this parameter and than we can delete it immidiately. 
*  After we apply delete operation
   *  when we apply selelct query than engine will go throgh log files and there have the info to not to consider paritcular files from the data files.

### **To read data present in checkpoint file**
* As parquet file is not human readable hence we have to create dataframe of that file to get and understand the info.
* To read the info present in the file.

In [ ]:
display(spark.read.format("parquet").load("dbfs:/delta/_delta_log/00000000000000000010.checkpoint.parquet"))

* The file contain the info of previous 10 operation done on the delta table. so with change in operation type on delta table the content of the checkpoint file will also change.
* it contain many column like add, remove, metadata, etc and engine using this info can get to know which daat file to include and which not to. thus info of all the 10 log file get in one single file and because of that the engine don't have to travese through each and every 10 log files.

### **How different operation works**

* **update** - E.g. update statement is given to delta lake table.
  * e.g. id=100 there will be 3 data files. Engine will scan all the data fiels or in the storage location and fetch the fiels where we are having suitable record to be updated.
  * out of all files lets say only 3 files identified for update than only these 3 fiels will be loaded to memory or ram.
  * Than in memeory the engine will scan the entire data in the files and make the required update.
  * Than it will wrtie the data back to storage system.

<hr>

## **create delta table**

**Execute commands in databricks**

### **Method 1 - Pyspark**

In [ ]:
from delta.tables import *

In [ ]:
DeltaTable.create(spark)\
    .tableName("shop")\
        .addColumn("shop_id","INT")\
            .addColumn("name","STRING")\
                .addColumn("sale","INT")\
                    .property("description",'table create for demo purpose')\
                        .location("shop/").execute()

* location
  * the location where this table should be created and if no value is given than it will create the table in hive meta store.
* description
  * In this we can mention the pupose of creating table or short note about table.

**If the table is already created or present than this command will give error**
* for this we can use craeteIfNotExist command.

In [ ]:
DeltaTable.createIfNotExists(spark)\
    .tableName("shop")\
        .addColumn("shop_id","INT")\
            .addColumn("name","STRING")\
                .addColumn("sale","INT")\
                    .property("description",'table create for demo purpose')\
                        .location("shop/").execute()

**if the tabel is already creaetd but we want to replace the table or table is not created**
* we can use createOrReplace
* If the table is not craeted than it will create.
* if table is present than it will drop the previous one and create the new one or create the new structure.

In [ ]:
DeltaTable.createOrReplace(spark)\
    .tableName("shop")\
        .addColumn("shop_id","INT")\
            .addColumn("name","STRING")\
                .addColumn("sale","INT")\
                    .property("description",'table create for demo purpose')\
                        .location("shop/").execute()

**In table name**
    * we can mention the database name also db.tab_name
    * default database is  - default
    * But we can mention the database name if want to store the table in other database.

**To check the new created table.**

In [ ]:
%sql
select * from shop;

## **Method 2 - Create delta table using SQL**

In [ ]:
%sql
CREATE TABLE shop (
shop_id INT,
name STRING,
sale INT,
) USING DELTA

**If table is already present than this can give error so use IF NOT EXIST**

In [ ]:
%sql
CREATE TABLE IF NOT EXIST shop (
shop_id INT,
name STRING,
sale INT,
) USING DELTA

**If table is already present in the system it will drop the previous version and recreate using the new syntax or structure**

%sql
CREATE OR REPLACE TABLE shop
(
shop_id INT,
name STRING,
sale INT,
) USING DELTA
LOCATION '/path'

* **path** 
    * location where we want to store the table
* **location**
  * it can be dbfs or s3 bucket etc.So this table will be created on top of thta external storage.

## **Method 3 -using existing dataframe**

**Create a dataframe**

In [20]:
shop_data= [(100,"virat",34), (200,"kohli",97),(100,"hunter",80)]

In [21]:
shop_schema= ["shop_id","name","sale"]

In [22]:
df= spark.createDataFrame(data=shop_data,schema=shop_schema)

In [23]:
df.show()

+-------+------+----+
|shop_id|  name|sale|
+-------+------+----+
|    100| virat|  34|
|    200| kohli|  97|
|    100|hunter|  80|
+-------+------+----+



### **Create table in metastore using dataframe schema and write data to it**

In [ ]:
df.write.fromat("delta").saveTable("shop")

* **format("delta")**
  * engine will understand that we need to create delta table
* saveTable("db.shop")
  * db - it is the name of the database in which you want to stor the table
  * shop - name of the table
  * In this we are creating the table but not defining the structure of the table
    * the structure, datatypes of the table will be taken from the stucture of datafarme from which the table is being created.
* In the previous methods we have created only structure of table and not inserted any data.
  * But when we create delta table using dataframe than the data present in the dataframe will be inserted into delta table own its own.

## **Delta table isntance**

**Delta table instance**
* To make soft link to actual delta table
* It will not create copy of the actual delta table.
* If any operation or changes done in actual table will be reflected in instance table.
* If any operation done on instance table than that will be reflected on orignal table.
* any operation we do with delta table or on its instance than it will be reflected in both.

Note -
  * Delta table instance is the replica of master table or actual table but it still pointing to same memory there is no copy created when we create instance.
  * If we only want to use pyspark or scala programming than we go for delta table instance creation.

**Usage of it**
* for applying dml query on delta table we are dependent on SQL query language.
* In some cases if we don't want to use sql query and want to use pyspark in that case we can use table instance.As we cannot use table name to exectue the operaion like we do in dataframe.
  
  **To create table instance***
  * using path - path -> where we have created the delta table.
    * > **DeltaTable.forPath(saprk,"/path/to/table")**
  * using table name
    * > **DeltaTable.forName(spark,"table_name")**

**First creating delta table using taht we will create delta table insatnce**
In real life project we generally create delta table uding the dataframe.

In [ ]:
DeltaTable.createIfNotExists(spark)\
    .tableName("shop")\
        .addColumn("shop_id","INT")\
            .addColumn("name","STRING")\
                .addColumn("sale","INT")\
                    .property("description",'table create for demo purpose')\
                        .location("/FileStore/shop/delta").execute()

**location("/FileStore/shop/delta")**
* Location where delta table will be created

**To insert some data to the delta table**

In [ ]:
%sql
insert into delta_internal_demo values(100,"philip",65);
insert into delta_internal_demo values(200,"Lara",68);

In [ ]:
%sql
SELECT * FROM shop;

### **To create delta table instance** 
suppose we don't want to use sql queries and use pysaprk operations in that we need to create delta table instance

**Method1 - using forPath**

In [ ]:
deltainstance1 = DeltaTable.forPath(spark,"/FileStore/shop/delta")

**/FileStore/shop/delta**
* location where delta table is present.

In [ ]:
deltaDF= deltainstance1.toDF()

**To see the dataframe created**

deltaDF.show()

**Sql vs pyspark commnad example-**
delete record

In [ ]:
%sql
delete from shop where shop_id=100

deltainstance1.delete("shop_id=100")

**Method 2 - using forName**

In [ ]:
deltainstance2 = DeltaTable.forName(spark,"shop")

* **shop**
    * shop is the name of the delta table
* **deltainstance2**
  * It is the delta table instance created.

**To create dataframe of delta instance**

In [ ]:
deltaDF= deltainstance2.toDF()

In [ ]:
deltaDF.show()

**sql vs pyspark operations**

In [ ]:
%sql 
DESCRIBE HISTORY shop

**This sql command will give the history of operation done on the shop table.**

deltainstance2.hoistory()

**pysaprk command to get the detail about history of operatoins on delta table instance or delta table**

## **Insert data into delta table**

### **Method 1 - SQL Style insert**

In [ ]:
%sql
INSERT INTO shop values ((100,"philip",65),(200,"Lara",68));

**To display the data from table**

In [ ]:
display(spark.sql("select * from shop"))

### **Method 2 - Dataframe insert** - insert data to delta table using dataframe.

In [ ]:
from pyspark.sql.types import StructField,StringType,StructType,IntegerType 

**To define the structure of the dataframe**

In [ ]:
shop_schema= StructType([\
    StructField("shop_id",IntegerType,True),\
    StructField("name",StringType,True),\
    StructField("sale",IntegerType,True)])

In [ ]:
shop_data= [(100,"virat",34), (200,"kohli",97),(100,"hunter",80)]

In [ ]:
df=spark.createDataFrame(data=shop_data,schema=shop_schema)

In [ ]:
df.show()

**To insert the dataframe into delta table using write command** 

In [ ]:
df.write.format("delta").mode("append").saveAsTable("shop")

* **mode**
  * append - if we want to retain prvious data than use append
  * overwrite - will delete the previous data store in table and than add the current data.
* **saveAsTable**
  * this is the table to which we want to store the data.
* **format** 
  * the format of the table to which we want to store the data.

**To insert the dataframe into delta table using insertInto command** 

In [ ]:
df.show()

In [ ]:
df.write.insertInto("shop",overwrite=False)

**insertInto("shop",overwrite=False)**
* shop - name of the delta tbale to which we want to insert the data.
* overwrite - 
  * false - to retain the previous data.
  * true - to overwrite the current data on previous data.

### **Method 3 - using Temp view**

In [ ]:
df.createOrReplaceTempView("delta_data")

**We create temporary view using the dataframe df and name of the view is called delta_data**

In [ ]:
%sql
select * from delta_data

This command is to query the temporary view created from df dataframe.

In [ ]:
%sql 
insert into shop
select * from delta_data

* **using above command we insert data into the shop delta table**
* delta_data
  * It is the temporary view created from df dataframe.
* shop 
  * it is the delta table to which we want to insert the data.


### **Method 4 - Spark SQL insert**

In [ ]:
spark.sql("insert into shop select * from delta_data")

<hr>

# **Delta table delete**

### **Method 1 - SQL**

In [ ]:
%sql 
delete from shop where shop_id=100;

### **Method 2 - SQL suing delta location**

In [ ]:
%sql 
delete from delta.'/delta/table/path/' where shop_id=100;

**/delta/table/path/** - location where delta table is present

### **Method 3 saprk sql

In [ ]:
spark.sql("delete from shop where shop_id=100;")

sql() - within this we can write the sql statement and we don't need to us the %sql or magic word

### **Method 4 - using delta table instance**

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forName(spark,"shop")

#deltaTable is the instance.

deltaTable.delete("shop_id=100")

## **update delta table**

### **Method 1 - using SQL standad table name**

In [ ]:
%sql 
update shop set name = "kartik" where shop_id =3;

### **Method 1 - using SQL standad table path**

In [ ]:
%sql 
update delta.'/delta/table/path/' set name = "kartik" where shop_id =3;

* **/delta/table/path/**
  * This is the location where delta table is present.

### **Method 4 - Pyspark standard using Table instance** - can use forPath or using froName

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark,"/delta/table/path/")

#declare the predicate by using a sql formatted string.

deltaTable.update(condition="shop_id=100", set={"salary":"15000"})

*  **deltaTable = DeltaTable.forPath(spark,"/delta/table/path/")**
   *  **spark** - SparkSession object
   *  **/delta/table/path/** - locatoin where delta table is present
   * **DeltaTable** - library,In PySpark, the DeltaTable class is part of the Delta Lake library, which extends Apache Spark to enable ACID transactions, schema evolution, and other features for data lakes. The DeltaTable API provides functionalities to interact with Delta tables, such as reading, writing, updating, and deleting data.
   * **deltaTable** - it is the delta table instance.

*  deltaTable.update()
   *  To update the instance
   *  condition="shop_id=100" - this is like filter condition to get the required rows which we want to update.
   *  set={"salary":"15000"} - this is the set condition to set the value.

## **merge(upsert) using pyspark and sql**

* upsert - update or insert if any match occur.

In [ ]:
shop_schema= StructType([\
    StructField("shop_id",IntegerType,True),\
    StructField("name",StringType,True),\
    StructField("sale",IntegerType,True)])

shop_data= [(100,"virat",34), (200,"kohli",97),(100,"hunter",80)]
# this is the source data



In [ ]:
df=spark.createDataFrame(data=shop_data,schema=shop_schema)

In [ ]:
df.show()

**Create delta lake table of name dim_employee**

In [ ]:
%sql 
CREATE OR REPLACE TABLE dim_shop (
    shop_id INT,
    name STRING,
    sale INT
) USING DELTA 
LOCATION "/delta/table/path/"


In [ ]:
%sql 
select * from dim_shop

* **As there is no record in the dim_shop thus it will fetch nothing.**
* dim_shop - is the target table. according to condition we will do insert or update into this table.


### **Method 1 - spark SQL**

In [ ]:
df.createOrReplaceTempView("source_view")

* df is the dataframe.
* from df we craete a temporary view called source_view


In [ ]:
%sql 
select * from source_view

In [ ]:
%sql 
select * from dim_shop

**Statement to execute merge operation**

In [ ]:
%sql 

MERGE INTO dim_shop as target
USING source_view as source
on target.shop_id=source.emp_id

WHEN MATCHED
THEN UPDATE SET

target.shop_id=source.shop_id
target.name=source.name
target.sales=source.sale

WHEN NOT MATCHED THEN 
INSERT (shop_id,name,sale) VALUES (shop_id,name,sale)


*  **target.shop_id=source.emp_id**
   *  when this condition is true than update the values with the new value from source
*  If conditoin fails than insert the record
*  taget- target is the delta table or dim_shop table
*  source - it is the dataframe from which we created the view called source_view from where we will insert the data.

### **Method 2 -  using pyspark**

**To create dataframe. This will be the source**

In [ ]:
shop_schema= StructType([\
    StructField("shop_id",IntegerType,True),\
    StructField("name",StringType,True),\
    StructField("sale",IntegerType,True)])

shop_data= [(100,"virat",34), (200,"kohli",97),(100,"hunter",80)]
# this is the source data


In [ ]:
df=spark.createDataFrame(data=shop_data,schema=shop_schema)

**To create the delta table. This will be our target table or **

In [ ]:
from delta.tables import *
delta_df=DeltaTable.forPath(spark,"/delta/table/path/")

* **/delta/table/path/**
  * This path contain the delta table
  * dim-shop - is the delta table name or target delta table

**pysaprk condition**

In [ ]:
delta_df.alias("target").merge(
source=df.alias("source"),
condition = "target.shop_id=source.shop_id").whenMatchedUpdate(set=
    {
    "shop_id":"source.shop_id",
    "name":"source.name",
    "sale":"source.sale"}
    ).whenNotMatchedInsert(
    values = {

        "shop_id":"source.shop_id", 
        "name":"source.name",
        "sale":"source.sale",
   }
).execute()

In [ ]:
%sql 
select * from dim_shop